In [33]:
import numpy as np
import random as rd
import math
import sys
import warnings

from scipy.optimize.optimize import _prepare_scalar_function
from scipy.optimize.optimize import vecnorm
from scipy.optimize.linesearch import (line_search_wolfe1, line_search_wolfe2,
                         line_search_wolfe2 as line_search,
                         LineSearchWarning)
from scipy.optimize.optimize import _line_search_wolfe12
from scipy.optimize.optimize import _LineSearchError
from scipy.optimize.optimize import OptimizeResult
from scipy.optimize.optimize import _status_message
from scipy.optimize._differentiable_functions import ScalarFunction, FD_METHODS

import IOHexperimenter as IOH
from IOHexperimenter import IOH_function
from zipfile import ZipFile
import os
from os.path import basename
import shutil
from shutil import make_archive

In [35]:
class BFGS:
    
    def __init__(self, budget):
        
        #parameter settings
        self.budget = budget
        self.d = 1
        self.gtol = 1e-5 #Gradient norm must be less than `gtol` before successful termination
        self.norm = np.inf #Order of norm (Inf is max, -Inf is min)
        self.disp = True
        self.eps = math.sqrt(np.finfo(float).eps) #If `jac is None` the absolute step size used for numerical approximation of the jacobian via forward differences.
        self.return_all = True
        self.jac = None
        self.finite_diff_rel_step = None
    
        
    def __call__(self, func):
        
        #Initialization
        self.d = func.number_of_variables
        evalcount = 0
        eval_budget = self.budget * self.d
        
        retall = self.return_all
      
        
        #initialize first point x0
        x0 = np.zeros(self.d)
        x_opt = None
        f_opt = np.inf
        for i in range (0, self.d):
            x0[i] = rd.uniform(-5,5)
            
        sf = _prepare_scalar_function(func, x0, self.jac, epsilon=self.eps,
                              finite_diff_rel_step=self.finite_diff_rel_step)
        
        f = sf.fun
        myfprime = sf.grad
        
        old_fval = f(x0)
        evalcount = evalcount +1
        gfk = myfprime(x0)
        
        k = 0
        N = len(x0)
        I = np.eye(N, dtype=int)
        Hk = I
        
        # Sets the initial step guess to dx ~ 1
        old_old_fval = old_fval + np.linalg.norm(gfk) / 2
        
        xk = x0
        if retall:
            allvecs = [x0]
        warnflag = 0
        gnorm = vecnorm(gfk, ord=self.norm)
        
        while (gnorm > self.gtol) and (evalcount < eval_budget) and (func.final_target_hit == False):
            
            pk = -np.dot(Hk, gfk)
            
            try:
                alpha_k, fc, gc, old_fval, old_old_fval, gfkp1 = \
                     _line_search_wolfe12(f, myfprime, xk, pk, gfk,
                                          old_fval, old_old_fval, amin=1e-100, amax=1e100)
            except _LineSearchError:
                # Line search failed to find a better solution.
                warnflag = 2
                break
                
            xkp1 = xk + alpha_k * pk
            if retall:
                allvecs.append(xkp1)
                
            sk = xkp1 - xk
            xk = xkp1
            if gfkp1 is None:
                gfkp1 = myfprime(xkp1)
                #evalcount increment here?
           
            yk = gfkp1 - gfk
            gfk = gfkp1
            k += 1
            
            if not np.isfinite(old_fval):
                # We correctly found +-Inf as optimal value, or something went
                # wrong.
                warnflag = 2
                break
            
            gnorm = vecnorm(gfk, ord=self.norm)
            if (gnorm <= self.gtol):
                break
            
            try:  # this was handled in numeric, let it remaines for more safety
                rhok = 1.0 / (np.dot(yk, sk))
            except ZeroDivisionError:
                rhok = 1000.0
            if self.disp:
                print("Divide-by-zero encountered: rhok assumed large")
            if np.isinf(rhok):  # this is patch for NumPy
                rhok = 1000.0
            if self.disp:
                print("Divide-by-zero encountered: rhok assumed large")
                
            A1 = I - sk[:, np.newaxis] * yk[np.newaxis, :] * rhok
            A2 = I - yk[:, np.newaxis] * sk[np.newaxis, :] * rhok
            Hk = np.dot(A1, np.dot(Hk, A2)) + (rhok * sk[:, np.newaxis] *
                                                     sk[np.newaxis, :])
            
            
        fval = old_fval

        if warnflag == 2:
            msg = _status_message['pr_loss']
        elif k >= eval_budget:
            warnflag = 1
            msg = _status_message['maxiter']
        elif np.isnan(gnorm) or np.isnan(fval) or np.isnan(xk).any():
            warnflag = 3
            msg = _status_message['nan']
        else:
            msg = _status_message['success']

        if self.disp:
            print("%s%s" % ("Warning: " if warnflag != 0 else "", msg))
            print("         Current function value: %f" % fval)
            print("         Iterations: %d" % k)
            print("         Function evaluations: %d" % sf.nfev)
            print("         Gradient evaluations: %d" % sf.ngev)


        result = OptimizeResult(fun=fval, jac=gfk, hess_inv=Hk, nfev=sf.nfev,
                        njev=sf.ngev, status=warnflag,
                        success=(warnflag == 0), message=msg, x=xk,
                        nit=k)

        if retall:
            result['allvecs'] = allvecs
        return result

In [36]:
fIDs = []
for fid in range (1,25):
    fIDs.append(fid)
    
from IOHexperimenter import IOHexperimenter
exp = IOHexperimenter()
exp.initialize_BBOB(fIDs,[1],[2,5,10], 1)
path = "/Users/dschroeder/Documents/Master Computer Science/Master_Thesis/code_and_data/experiments/"
experiment_ID = input("enter date-time, format ddmm-hhmm")
experiment_name =  "BFGS/experiment" + experiment_ID
exp.set_logger_location(path + experiment_name, "run")
exp([BFGS(1000000)])
#create a ZIP file for IOHprofiler upload
shutil.make_archive(path + experiment_name,"zip", path + experiment_name)

enter date-time, format ddmm-hhmm 0809-1745


Running single-threaded
Divide-by-zero encountered: rhok assumed large
Divide-by-zero encountered: rhok assumed large
Divide-by-zero encountered: rhok assumed large
Divide-by-zero encountered: rhok assumed large
Optimization terminated successfully.
         Current function value: 79.480000
         Iterations: 3
         Function evaluations: 12
         Gradient evaluations: 4
1 12 3.80929179074603e-12
Divide-by-zero encountered: rhok assumed large
Divide-by-zero encountered: rhok assumed large
Divide-by-zero encountered: rhok assumed large
Divide-by-zero encountered: rhok assumed large
Divide-by-zero encountered: rhok assumed large
Divide-by-zero encountered: rhok assumed large
Optimization terminated successfully.
         Current function value: 79.480000
         Iterations: 3
         Function evaluations: 24
         Gradient evaluations: 4
1 24 3.8276719952291546e-11
Divide-by-zero encountered: rhok assumed large
Divide-by-zero encountered: rhok assumed large
Divide-by-zero e

'/Users/dschroeder/Documents/Master Computer Science/Master_Thesis/code_and_data/experiments/BFGS/experiment0809-1745.zip'

In [6]:
_epsilon = math.sqrt(np.finfo(float).eps)